In [15]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [17]:
URL = "https://cfd.nu.edu.pk/all-departments/"
cfd_headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"}

In [19]:
r = requests.get(URL,headers=cfd_headers)
soup = BeautifulSoup(r.content, 'html.parser')

In [21]:
print(soup.prettify())

<!DOCTYPE html>
<html lang="en-US">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <link href="https://gmpg.org/xfn/11" rel="profile"/>
  <title>
   All Departments – FAST NUCES | CFD
  </title>
  <meta content="max-image-preview:large" name="robots">
   <link href="//fonts.googleapis.com" rel="dns-prefetch">
    <link href="//s.w.org" rel="dns-prefetch"/>
    <link href="https://cfd.nu.edu.pk/feed/" rel="alternate" title="FAST NUCES | CFD » Feed" type="application/rss+xml"/>
    <link href="https://cfd.nu.edu.pk/comments/feed/" rel="alternate" title="FAST NUCES | CFD » Comments Feed" type="application/rss+xml"/>
    <script type="text/javascript">
     window._wpemojiSettings = {"baseUrl":"https:\/\/s.w.org\/images\/core\/emoji\/13.1.0\/72x72\/","ext":".png","svgUrl":"https:\/\/s.w.org\/images\/core\/emoji\/13.1.0\/svg\/","svgExt":".svg","source":{"concatemoji":"https:\/\/cfd.nu.edu.pk\/wp-includes\/js\/wp-emoji-release.min.

In [23]:
deptLinks = []
for link in soup.select("ul.sub-menu > li > a"): 
    deptURL = link.get("href")
    if deptURL and "/department-" in deptURL and "#" not in deptURL: 
        deptLinks.append(deptURL)


deptLinks.append('https://cfd.nu.edu.pk/staff/')
deptLinks = list(set(deptLinks))
print(deptLinks)

['https://cfd.nu.edu.pk/department-fsm/', 'https://cfd.nu.edu.pk/department-se/', 'https://cfd.nu.edu.pk/department-ai/', 'https://cfd.nu.edu.pk/staff/', 'https://cfd.nu.edu.pk/department-cs/', 'https://cfd.nu.edu.pk/department-ee/', 'https://cfd.nu.edu.pk/department-sh/']


In [85]:
ID = []                    # Integer
Name = []                  # String
Designation = []           # String
HEC_Approved_PHD_Supervisor = []  # Bool
Highest_Education = []     # String
Email = []                 # EMAIL
Department = []            # String
Extension = []             # Integer
ImageURL = []              # URL

In [87]:
teacherProfileLinks = []
for link in deptLinks:
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
    }
    r = requests.get(link, headers=headers)
    soup = BeautifulSoup(r.content, 'html.parser')
    for teacher in soup.find_all("div", class_="unitech-teacher__inner"):
        name = teacher.find("h4").find("a").text.strip()
        Name.append(name)
        
    for teacher in soup.find_all("div", class_="unitech-teacher__hover__info"):
        email = teacher.find("p")
        Email.append(email.text.strip() if email else "N/A")
        
    for teacher in soup.find_all("div", class_="unitech-teacher__inner"):
        designation = teacher.find("h6")
        Designation.append(designation.text.strip())
        
    for teacher in soup.find_all("div", class_="unitech-teacher__hover__info"):
        isApproved = teacher.find("p", class_="hec")
        if isApproved:
            HEC_Approved_PHD_Supervisor.append(True)
        else:
            HEC_Approved_PHD_Supervisor.append(False)
            
    for teacher in soup.find_all("div", class_="unitech-teacher__thumb"):
        image = teacher.find("img")
        ImageURL.append(image.get("src", "").strip())
    
    for teacher in soup.find_all("div", class_="unitech-teacher__inner"):
        teacherName = teacher.find("h4").find("a")
        teacherProfiles = teacherName.get("href", " ").strip()
        teacherProfileLinks.append(teacherProfiles)
        
# print(Name)
# print(Email)
# print(Designation)
# print(HEC_Approved_PHD_Supervisor)
# print(ImageURL)
# print(teacherProfileLinks)

print(len(Name))
print(len(Email))
print(len(Designation))
print(len(HEC_Approved_PHD_Supervisor))
print(len(ImageURL))
print(len(teacherProfileLinks))

194
194
194
194
194
194


In [89]:
for profLink in teacherProfileLinks:
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
    }
    r = requests.get(profLink, headers=headers)
    soup = BeautifulSoup(r.content, 'html.parser')

    dept = "N/A"
    ext = "N/A"
    edu = "N/A"
    address = soup.find("ul", class_="teacher__address")
    info = soup.find("ul", class_="teacher__address")
    education = soup.find("div", class_="htc__skill__container")
    
    ID = list(range(len(teacherProfileLinks)))

    if address:
        li = address.find("li") 
        if li:
            dept = li.text.strip()
    Department.append(dept)
    
    if info:
        for li in info.find_all("li"):
            if "Ext:" in li.text:
                extension = li.text.split("Ext:")[-1].strip()
                if extension.isdigit():
                    ext = int(extension) 
                break
    Extension.append(ext)

    if education:
        teacherEducation = education.find("p")
        if teacherEducation:
            edu = teacherEducation.text.strip()
        else:
            educationList = education.find("ul")
            if educationList:
                deg = educationList.find("li")  
                if deg:
                    edu = deg.text.strip()

    if "," in edu:
        edu = edu.split(",")[0]

    Highest_Education.append(edu)

# print(Department)
print(len(Department))
# print(Extension)
print(len(Extension))
# print(Highest_Education)
print(len(Highest_Education))
# print(ID)
print(len(ID))

194
194
194
194


In [91]:
faculty_df = pd.DataFrame({'ID': ID , 'Name': Name , 'Designation': Designation , 'HEC_Approved_PHD_Supervisor': HEC_Approved_PHD_Supervisor ,
                       'Highest_Education': Highest_Education , 'Email' : Email , 'Department': Department , 'Extension': Extension ,
                       'ImageURL': ImageURL })

In [99]:
faculty_df.to_csv('chin.csv', index=False)

In [97]:
print("Names:", Name)
print("Emails:", Email)
print("Designations:", Designation)
print("Departments:", Department)
print("Image URLs:", ImageURL)

Names: ['Dr. Gulfam Haider', 'Dr. Muhammad Rafiq', 'Dr. Misbah Hayat', 'Dr. Saifullah', 'Mr. Syed Zeeshan', 'Mr. Ahmad Salah', 'Ms. Amna Babar', 'Mr. Hafiz Muhammad', 'Mr. Muhammad Nasif', 'Dr. Qamar uz', 'Dr. Affan Rauf', 'Dr. Amer Rasheed', 'Dr. Shamsa Abid', 'Dr. Shanza Abbas', 'Ms. Anmol Bilal', 'Mr. Awais Azam', 'Mr. Fraz Zaheer', 'Mr. Jawad Khalid', 'Ms. Kanwal Naz', 'Ms. Naba Rahim', 'Ms. Sahar Ajmal', 'Ms. Samia Aziz', 'Mr. Saqib Ameer', 'Ms. Tehreem Aslam', 'Mr. Umer Iqbal', 'Dr. Hashim Yasin', 'Dr. Anwar Shah', 'Mr. Asif Ameer', 'Mr. Hamza Yousuf', 'Mr. Muhammad Yousaf', 'Ms. Rukhsana Zafar', 'Mr. Tahir Farooq', 'Mr. Waseem Ahmad', 'Mr. Muhammad Usman Naeem', 'Ms. Aysha Shafiq', 'Mr. M. Atif Nadeem', 'Mr. Muhammad Shahid Iqbal', 'Mr. Muhammad Usman', 'Mr. Naveed Ahmed', 'Ms. Nooria Fatima', 'Mr. Asad Ali', 'Mr. Muhammad Khalid', 'Ms. Zunaira Ashiq', 'Ms. Maria Intizar', 'Mr. Safdar Abbas', 'Mr. Javaid Iqbal', 'Mr. Muhammad Khalid', 'Ms. Afshan Rani', 'Mr. Ghulam Shabbir', 'Mr